In [27]:
# Imports
import numpy as np
import pandas as pd

In [28]:
# Read in the data
df = pd.read_csv('./Data/small_box_office_flat_file.csv', header=0)

In [29]:
# Set boolean values to 'bool' dtype
column_names = df.select_dtypes(include=[np.number]).columns 

# re-extracting the columns of numerical type (using awesome np.number1 :)) then getting the max of those and storing them in a temporary variable m.
m=df[df.select_dtypes(include=[np.number]).columns].max().reset_index(name='max')

# I then did a filter like BrenBarn showed in another post to extract the rows which had the max == 1 and stored it in a temporary variable n.
n=m.loc[m['max']==1, 'max']

# I then extracted the indexes of the rows from n and stored them in temporary variable p.
# These indexes are the same as the indexes from my original dataframe 'df'.
p=column_names[n.index]

df[p] = df[p].astype(bool)

In [30]:
# Convert Categorial colums to 'category' dtype
cat_column_names = ['OriginLang', 'CatRuntime', 'Genre']
df[cat_column_names] = df[cat_column_names].astype('category')

In [34]:
# Convert 'ReleaseDate' to datetime
df['ReleaseDate'] = pd.to_datetime(df['ReleaseDate'], format='%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   ID                         1000 non-null   int64         
 1   Budget                     1000 non-null   int64         
 2   OriginLang                 1000 non-null   category      
 3   Popularity                 1000 non-null   float64       
 4   ReleaseDate                1000 non-null   datetime64[ns]
 5   Runtime                    1000 non-null   int64         
 6   CatRuntime                 1000 non-null   category      
 7   IsEnOriginalLang           1000 non-null   bool          
 8   HasWebPage                 1000 non-null   bool          
 9   HasPoster                  1000 non-null   bool          
 10  HasTagline                 1000 non-null   bool          
 11  ReleaseYear                1000 non-null   int64         
 12  Release